# Generative models
In generative models, the assumption is that the data comes from an unknown distribution $P_r$. The goal of the generative model is to learn a distribution $P_\theta$ that approximates $P_r$, where $\theta$ are the parameters of the distribution.

The approach involves learning a function that transforms an existing distribution $Z$ into $P_\theta$. 

Given the function $P_\theta$, the MLE objective is given by 

\begin{equation}
\max\limits_{\theta \epsilon R^d} \frac{1}{m} \sum\limits_{i=1}^{m} log P_\theta (x^{(i)})
\end{equation}

Ref:
https://www.alexirpan.com/2017/02/22/wasserstein-gan.html

## Kullback-Leibler and Jensen-Shannon Divergence (Performance metrics)

Kullback-Leibler divergence measures how one probability distribution $P_{dist}$ diverges or how similar it is to another probability distribution $Q_{dist}$. 
\begin{equation}
D_{KL}(P_{dist}||Q_{dist}) = \int_{x} P_{dist}(x) \log \frac{P_{dist}}{Q_{dist}}dx
\end{equation}

The divergence is zero when both the distributions are equal. The problem with KL divergence is that it is asymmetric. See code below...

In [37]:
actual = np.array([0.4, 0.6]) 
model1 = np.array([0.2, 0.8]) 
model2 = np.array([0.35, 0.65]) 

# Find similarity between the two distribution using KL divergence
from scipy.stats import entropy
import numpy as np

kl1 = (model1 * np.log(model1/model2)).sum()
print('Kullback Leibler Divergence Computed = ',kl1)

KL_divergence = entropy(np.abs(model1),np.abs(model2))
print('Kullback Leibler Divergence = ',KL_divergence)

Kullback Leibler Divergence Computed =  0.0541883342355
Kullback Leibler Divergence =  0.0541883342355


In [39]:
kl2 = (model2 * np.log(model2/model1)).sum()
print('Kullback Leibler Divergence Computed = ',kl2)

KL_divergence_reverse = entropy(np.abs(model2),np.abs(model1))
print('Kullback Leibler Divergence Reverse = ',KL_divergence_reverse)

Kullback Leibler Divergence Computed =  0.0608999386715
Kullback Leibler Divergence Reverse =  0.0608999386715


Note: The divergence is not bidirectional value!

## Jensen-Shannon Divergence
This divergence distribution is symmetric and smooth. Uses the 2 KL divergences between the distributions to generate the JS divergence.
\begin{equation}
D_{JS}(P_{dist}||Q_{dist}) = \frac{1}{2}D_{KL}(P_{dist}||\frac{P_{dist}+Q_{dist}}{2}) + \frac{1}{2}D_{KL}(Q_{dist}||\frac{P_{dist}+Q_{dist}}{2})
\end{equation}

In [41]:
model3 = (model1+model2)/2
dkl1 = (model1*np.log(model1/model3)).sum()
dkl2 = (model2*np.log(model2/model3)).sum()
js1 = 0.5*dkl1 + 0.5*dkl2
print("Jensen-Shannon Divergence Computed = ",js1)

js_divergence = 0.5*entropy(model1,model3) + 0.5*entropy(model2,model3)
print("Jensen-Shannon Divergence = ",js_divergence)

Jensen-Shannon Divergence Computed =  0.0142442460678
Jensen-Shannon Divergence =  0.0142442460678


Therefore, for GAN we use the Jensen-Shannon divergence metric to compare the two distributions.

## GAN
GAN consists of two models, 

* Discriminator $D$, which estimates the probability of a given sample coming from the real dataset. It also works as a critic and is optimized to discriminate if any fake sample arrive.  
* Generator $G$, which generates synthetic samples given a noisy input $z$. It tries to generate samples closer to the real dataset and in turns tries to outfox the discriminator. 
![title](https://lilianweng.github.io/lil-log/assets/images/GAN.png)

The two models competing against each other creates a Zero-Sum game also called as Nash equilibrium. 
From wikipedia: 


### In terms of game theory, if each player has chosen a strategy, and no player can benefit by changing strategies while the other players keep theirs unchanged, then the current set of strategy choices and their corresponding payoffs constitutes a Nash equilibrium.

The goal of the discriminator is to 
1. classify the real data accurately by maximizing $log(D(x))$ , 
2. classify the fake data accurately as fake by maximizing $log(1-D(G(z)))$.

The goal of the generator is to, 
1. Generate good fake examples to outwit the discriminator by minimizing $log(1-D(G(z)))$.

When combined they create a minmax game where the loss function looks the following, 
\begin{align}
\min_{G}\max_{D}L(D,G)&=E_{x~p_r(x)}[\log D(x)] + E_{z~p_z(z)}[\log(1-D(G(z)))]\\
                     &=E_{x~p_r(x)}[\log D(x)] + E_{z~p_z(z)}[\log(1-D(x))]
                     \end{align}
                     
$E_{x~p_r(x)}[log D(x)]$ has no impact on $G$ during gradient descent updates!

## Optimal value for $D$ and the global optimal: 
Given the loss function as following: 
\begin{equation}
L(G,D) = \int_{x}(p_r(x)\log(D(x)) + p_g(x)log(1-D(x)))dx
\end{equation}, 
### the best value for the discriminator is 
\begin{equation}
D^*(x)=x^{~*} = \frac{p_r(x)}{p_r(x)+p_g(x)} \epsilon [0,1]
\end{equation}
Once the generator is trained to its optimal, $p_g$ gets very close to $p_r$. when $p_g=p_r, D^*(x)$ becomes $1/2$

### When both $G$ and $D$ are at their optimal values, 
we have $p_g=p_r$ and $D^*(x)=1/2$ and the loss function becomes: 
\begin{equation}
L(G,D^*)=-2\log 2
\end{equation}

### The JS divergence thus becomes, 
$L(G,D^*)=2D_{JS}(p_r||p_g)-2\log 2$
Essentially the loss function of GAN quantifies the similarity between the generative data distribution $p_g$ and the real sample distribution $p_r$ by JS divergence when the discriminator is optimal. 

check: https://lilianweng.github.io/lil-log/2017/08/20/from-GAN-to-WGAN.html#kullbackleibler-and-jensenshannon-divergence

## Problems: 
1. GANs are hard to train and to achieve nash equilibrium in a two-player non-cooperative game. 
2. Vanishing Gradient problem with the discriminator. 
3. Mode collapse: During the training, the generator may collapse to a setting where it always produces same outputs. 
4. Lack of proper evaluation metric. 


## Possible Improvements
1. Feature matching: Feature matching suggests to optimize the discriminator to inspect whether the generator’s output matches expected statistics of the real samples.
2. Minibatch Discrimination: With minibatch discrimination, the discriminator is able to digest the relationship between training data points in one batch, instead of processing each point independently.
3. Historical Averaging: For both models add $||\Theta - \frac{1}{t}\sum_{i=1}^t \Theta||^2$ into the loss function, where $\Theta$ is the model parameter and $\Theta_i$ is how the parameter is configured at the past training time $i$. 
4. One-sided Label Smoothing: Use softened label values such 0.1 , 0.9 instead of hard labels such as 0 and 1. 
5. Virtual Batch Normalization: Each data sample is normalized based on a fixed or reference batch instead of the minibatch. 
6. Adding Noises
7. Better Metric. eg. Wassenstein Metric

Ref: 
1. https://lilianweng.github.io/lil-log/2017/08/20/from-GAN-to-WGAN.html#kullbackleibler-and-jensenshannon-divergence
2. http://papers.nips.cc/paper/6125-improved-techniques-for-training-gans.pdf
3. https://arxiv.org/pdf/1701.04862.pdf
4. https://arxiv.org/pdf/1701.07875.pdf


# Wasserstein distance or Earth Movers distance
Wasserstein distance is a measure of the distance between two probability distributions. It is also called Earth Movers distance, because informally it can be interpreted as the minimum energy cost of moving and transforming a pile of dirt in the shape of one probability distribution to the shape of the another distribution. The cost: the amount of dirt moved x the moving distance. 
Hence, the distance formula becomes: 
\begin{equation}
W(p_r, p_g) = \inf_{\gamma \sim \Pi(p_r, p_g)} \mathbb{E}_{(x, y) \sim \gamma}[\| x-y \|]
\end{equation}. 
In the formula above, $\Pi(p_r, p_g)$ is the set of all possible joint probability distributions between $p_r$ and $p_g$. One joint distribution $\gamma \in \Pi(p_r, p_g)$ describes one dirt transport plan. Precisely $\gamma(x, y)$ states the percentage of dirt should be transported from point $x$ and $y$ as as to make $x$ follows the same probability distribution of $y$. 
#### Check the references to know, why wasserstein distance is better than JS divergence.
Ref: 
1. https://lilianweng.github.io/lil-log/2017/08/20/from-GAN-to-WGAN.html#kullbackleibler-and-jensenshannon-divergence
2. https://medium.com/@jonathan_hui/gan-wasserstein-gan-wgan-gp-6a1a2aa1b490 

## Wasserstein distance as GAN loss function
It is intractable to exhause all the possible joint distributions in $\Pi(p_r, p_g)$ to compute $\inf_{\gamma \sim \Pi(p_r, p_g)}$. Thus the proposal of the following formula based on the Kantorovich-Rubinstein duality: 
\begin{equation}
W(p_r, p_g) = \frac{1}{K} \sup_{\| f \|_L \leq K} \mathbb{E}_{x \sim p_r}[f(x)] - \mathbb{E}_{x \sim p_g}[f(x)]
\end{equation}
where $\sup$ (supremum) is the opposite of $\inf$ (infimum); we want to measure the least upper bound or in other words, the maximum value. 
$f$ is a 1-Lipschitz function following the constraint, 
\begin{equation}
\lvert f(x_1) - f(x_2) \rvert \leq K \lvert x_1 - x_2 \rvert
\end{equation}

So to calculate the Wasserstein distance, we just need to find a 1-Lipschitz function. Like other deep learning problem, we can build a deep network to learn it. Indeed, this network is very similar to the discriminator D, just without the sigmoid function and outputs a scalar score rather than a probability. This score can be interpreted as how real the input images are. In reinforcement learning, we call it the value function which measures how good a state (the input) is. We rename the discriminator to critic to reflect its new role. Let’s show GAN and WGAN side-by-side.

Thus the “discriminator” is not a direct critic of telling the fake samples apart from the real ones anymore. Instead, it is trained to learn a K-Lipschitz continuous function to help compute Wasserstein distance. As the loss function decreases in the training, the Wasserstein distance gets smaller and the generator model’s output grows closer to the real data distribution.

![title](https://lilianweng.github.io/lil-log/assets/images/WGAN_algorithm.png)

Compared to the original GAN algorithm, the WGAN undertakes the following changes:

* After every gradient update on the critic function, clamp the weights to a small fixed range, [−c,c].
* Use a new loss function derived from the Wasserstein distance, no logarithm anymore. The “discriminator” model does not play as a direct critic but a helper for estimating the Wasserstein metric between real and generated data distribution.
* Empirically the authors recommended RMSProp optimizer on the critic, rather than a momentum based optimizer such as Adam which could cause instability in the model training.

### WGAN example results
For implementation check: 

## Real Images
![title](GAN_data/Real_images.png)

WGAN generated Images
![title](GAN_data/GAN_images.png)